# Assignment 7

### Part A

In [29]:
import numpy as np
import pandas as pd

Consider the following way of representing a portfolio. Each row in the DataFrame represents one security, with information given about the security's beta, volatility, and amount invested in that security. Note that there is not guaranteed to be 2 securities in the portfolio.

In [30]:
portfolio = pd.DataFrame(data = {"Beta": [1.2, 1.4], 
                                 "Volatility": [0.25, 0.4],
                                 "Amount Invested": [4000, 6000] })



portfolio

Beta  Volatility  Amount Invested
0   1.2        0.25             4000
1   1.4        0.40             6000

Write the function ```expected_return```, which calculates the expected return of the portfolio ```P```. There is a risk free asset with risk free rate ```rf```, and the market risk premium is ```mkt_rp```. 

expected return: expected_return(portfolio, 0.02, 0.05) should yield 0.086. 

In [31]:
def expected_return(P, rf, mkt_rp):

    total_portfolio_val = sum(P['Amount Invested'])

    P['Expected'] = (rf + P['Beta'] * mkt_rp) * (P['Amount Invested'] / total_portfolio_val)

    return sum(P['Expected'])

expected_return(portfolio, 0.02, 0.05)

0.086

Write the function ```volatility```, which calculates the volatility of the portfolio. You can assume that the correlation between all (different) pairs of stocks in the portfolio is ```cor```.

expected return: volatility(portfolio, -0.2) should yield 0.2408318915758459

In [43]:
def volatility(P, cor):

    P['Weight'] = P['Amount Invested'] / sum(P['Amount Invested'])
    ex2 = (P['Weight'] ** 2) * (P['Volatility'] ** 2)
    prods = P.prod(axis=0)
    var = sum(ex2) + (2 * prods['Weight'] * prods['Volatility'] * cor)
    
    return var ** (1/2)

volatility(portfolio, -0.2)

0.24083189157584595

Suppose there is a risk-free asset with return of ```rf```, the market risk premium is ```mkt_rp```, and the volatility of returns on the market is ```mkt_vol```.

Write the function ```expected_return2```, which creates a portfolio with volatility ```vol``` using (only) the risk free asset and the market portfolio, and calculates the expected return of that portfolio.

expected return: expected_return2(0.02, 0.05, 0.16, 0.2408) should yield 0.09525

In [48]:
def expected_return2(rf, mkt_rp, mkt_vol, vol):

    vol_mult = vol / mkt_vol

    return rf + vol_mult * mkt_rp

expected_return2(0.02, 0.05, 0.16, 0.2408)


0.09525

### Part B

Write the function ```CAPM_beta```, which will consume a **filepath** ```return_path```, (formatted as the provided "monthlycsrp_mod7.csv"), and a **filepath** `factors_path` (formatted as the provided "F-F_Research_Data_Factors.CSV"). This function will return a DataFrame formatted like the one supplied. Each row in the returned DataFrame should correspond with 1 CUSIP, its expected return, and its CAPM-beta.

Please note that monthlycsrp_mod7.csv's return data are in decimals. "F-F_Research_Data_Factors.CSV" has data in percentages. More details of "F-F_Research_Data_Factors.CSV" are available on https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html. 

Also, note that firm i's CAPM-beta is calculated as Cov(R_i-Rf, R_mkt – Rf)/Var(R_mkt-Rf). 

expected return: it should return output of 3225 rows × 3 columns

$ \text{Expected Return} = R_f + [\beta * \text{Market Return Premium}] \\ \\
\beta = \frac{Cov(R_i - R_f, R_{mkt} - R_f)}{Var(R_{mkt} - R_f)} $

In [100]:
def CAPM_beta(return_path, factors_path):

    data = pd.read_csv(return_path)
    factors = pd.read_csv(factors_path)

    data['RET'] = data['RET'].astype(float)
    data['month'] = data['month'].map("{:02}".format)
    data['date'] = data['year'].astype(str) + data['month']

    factors['Mkt-RF'] /= 100
    factors['RF'] /= 100
    factors['date'] = factors['date'].astype(str)

    merged = pd.merge(data, factors, on='date')

    dict = {
        "CUSIP" : [] ,
        "EXP_RETURN" : [],
        "BETA" : [],
    }

    merged['Ri-RF'] = merged['RET'] - merged['RF']

    grouped = merged.groupby(['CUSIP'], group_keys = False)

    for name, group in grouped:

        cov = np.cov(group['Ri-RF'], group['Mkt-RF'])[0][1]
        var = group['Mkt-RF'].var()
        beta = cov / var

        exp_return = group['RET'].mean()

        dict['CUSIP'].append(group.iloc[0]['CUSIP'])
        dict['BETA'].append(beta)
        dict['EXP_RETURN'].append(exp_return)

    output = pd.DataFrame(data = dict)
    
    return output

output = CAPM_beta("monthlycrsp_mod7.csv", "F-F_Research_Data_Factors.csv")
output

CUSIP  EXP_RETURN      BETA
0     00081T10    0.016252  2.591073
1     00108M10    0.011101  0.741057
2     00130H10    0.016152  1.457100
3     00163U10    0.015538  0.975530
4     00182C10    0.014772  1.406085
...        ...         ...       ...
3220   9959910    0.009933  0.763107
3221   9972410    0.012866  1.280548
3222   9972510    0.017177  1.065298
3223   9984910    0.009006  1.876418
3224   9985510    0.015699  1.166459

[3225 rows x 3 columns]

Write the function ```slope```, which will return the slope of the security market line. This function will take in a **DataFrame** ```data```, whose format is that of the output of ```CAPM_beta```. You may find the function `np.polyfit` useful.

In [99]:
def slope(data):
    
    # This should return 0.007981588120535642... with the provided data.
    return np.polyfit(data['BETA'], data['EXP_RETURN'], 1)[1]

slope(output)

0.007981588120535638